# Group Notebook - Condor

## Import Modules

In [1]:
import math
import csv
import urllib.request
import json

# Create Daily Static Objects

1. Create country currency object which contains country_name : currency_code Dictionary
2. Create todays currency rates object which contains currency_code : todays_rate Dictionary
3. Initialize an empty atlas object. Dictionaries of this object will only fill by demand.

    There are 3 dictionaries within the Atlas class:


    Airport Dictionary: Contains a dictionary of airport OBJECTS as values to the airport.code as the key 
    Distance "Matrix" in dictionary format
    Weighted "Matrix" product of distance matrix*todays currency rates for the departing airport

        
4. Create the aircraft roster OBJECT and create all the aircraft OBJECTS to fill the roster.aircraft_dict

### country_currency OBJECT - Country Name(K) : Currency Code (V) Dictionary

In [2]:
#Creating a dictionary of country names to the currency code from file "itu.csv"
#Dictionary of key "Country" : value "CurrencyCode"
#Static data that does not change hence ???not implemented as a class
class country_currency():
    
    def __init__(self, date):
        self.date = date
    try:    
        country_currency_code_dict = {}
        with open('countrycurrency.csv') as csvfile:
            readCSV = csv.reader(csvfile, delimiter=',')
            csvfile.readline()   # skip the first line
            for row in readCSV:
                country_currency_code_dict[row[0]] = row[14]
    except FileNotFoundError as e:
        print("countrycurrency.csv file not found in current directory")

### todays_currency_rates OBJECT - Currency Code(K) : Todays Rates(V)  Dictionary

In [3]:
#Creating a currency rate class for that day (optional as a class or global dict?)
#Using an API to get daily rates
#Dictionary of key "CurrencyCode" : value "TodaysExchangeRate"
class todays_currency_rates():
    
    def __init__(self, date):
        self.date = date
        
    currency_code_rates_dict = {}
    with urllib.request.urlopen("http://data.fixer.io/api/latest?access_key=340fa686ffcff7fdb67e23b57b246b8a") as url:
        data = json.loads(url.read().decode())
        for currency in data['rates']:
            currency_code_rates_dict[currency] = data['rates'][currency]
    
    #If there are any currencies that the API does not pick up then add as key from the CSV file provided
    try:
        with open('currencyrates.csv') as csvfile:
            readCSV = csv.reader(csvfile, delimiter=',')
            csvfile.readline()   # skip the first line
            for row in readCSV:
                if row[1] not in currency_code_rates_dict:
                    currency_code_rates_dict[row[1]] = float(row[3])
    except FileNotFoundError as e:
        print("countrycurrency.csv file not found in current directory")

### Airport OBJECT

In [4]:
#Airport class that will be created by the Atlas class once the file of airports is provided
class Airport():
    def __init__(self, country_currency, todays_currency_rates, code="", name="", country="", lat=0, long=0):
        self.set_code(code)
        self.set_name(name)
        self.set_country(country)
        self.set_long(long)
        self.set_lat(lat)
        self.set_radlong(long)
        self.set_radlat(lat)
        self.set_currencyCode()
    
    def set_code(self, code):
        
        codeList=[]
     
        csv_file = csv.reader(open('airport.csv', 'r'), delimiter=",")
        for row in csv_file:
            codeList.append(str(row[4]))
        
        if code not in codeList:
            print("Error- code not valid")
        else:
            self._code = code
    
    def set_name(self, name):
         #No error checking for name as name may contain abbreviations. Also code will uniquely identify airports rather than name     
        self._name=name
    
    def set_country(self,country):
        if country not in country_currency.country_currency_code_dict:
            print("Error- country not valid")
        else:
            self._country = country
    
    def set_long(self,long):
        
        if long >180 or long <-180:
            print("Invalid longitude entered")
        else:
            self._long = long
            
    
    def set_lat(self,lat):
        
        if lat >90 or lat <-90:
            print("Invalid latitude entered")
        else:
            self._lat = lat
           
    
    def set_radlong(self,long):
        self._radlong = self.radians(long)
        
    def set_radlat(self,lat):
         self._radlat = self.radians(lat)    
    
    def set_currencyCode(self):
        try:
            #Index the country currency code dict using the country as the key
            self._currencyCode = country_currency.country_currency_code_dict[self.get_country()]
        except:
            #If currency is not specified automatically default to USD
            self._currencyCode = country_currency.country_currency_code_dict["United States"]
    
    
    def get_code(self):
        return self._code
    
    def get_name(self):
        return self._name
    
    def get_country(self):
        return self._country
    
    def get_long(self):
        return self._long
    
    def get_lat(self):
        return self._lat
    
    def get_radlong(self):
        return self._radlong
    
    def get_radlat(self):
        return self._radlat
    
    def get_currencyCode(self):
        return self._currencyCode
    
    
    #Method that returns the absolute Great Circular Distance between two airports
    def distance_between(self, airport_code):
        #Getting radian equivalent of 90 degrees
        rad90 = self.radians(90)
        
        
        argument_to_acos = (math.sin(rad90-self.get_radlat())*math.sin(rad90-airport_code.get_radlat())*\
        math.cos(self.get_radlong()-airport_code.get_radlong()))+(math.cos(rad90-self.get_radlat())*\
        math.cos(rad90-airport_code.get_radlat()))
        
        #To take care of floating point number 1.0000000002... Cannot get math.acos of > 1!
        if argument_to_acos > 1:
            argument_to_acos = 1
        distance = (math.acos(argument_to_acos))*6371
        return round(abs(distance), 0)
    
    #method that will return the actual numeric value of the currency at todays rate
    #indexes the Today instance of CurrencyRate class using the currencyCode
    def get_currencyToday(self):
        try:
            #Index the todays currency rates dict using the currency code as the key
            return todays_currency_rates.currency_code_rates_dict[self.get_currencyCode()]
        except:
            #If the currency code does not exist on the API or the CSV file then return the USD as the currency
            return todays_currency_rates.currency_code_rates_dict["USD"]
        
    def radians(self, degrees):
        radians = degrees*((math.pi)/180)
        return radians

### Atlas OBJECT

In [5]:
class atlas():
    __airport_dict = {}
    __distance_matrix = {}
    __weighted_matrix = {}
    
    def __init__(self, country_currency, todays_currency_rates, name):
        self.name = name
    
    @classmethod
    def update(cls,route_list):
        
        #Airport Dict
        
        #Check that each airport in route has been created as an airport object if not create and add to dict
        try:
            csv_file = csv.reader(open('airport.csv', 'r'), delimiter=",")
        except FileNotFoundError as e:
            return("airport.csv file not found in current directory")
        else:
            for airport_code in route_list:
                #If airport_code not in dictionary then create an airport object and add airport_code as key to dict
                if airport_code not in cls.get_airport_dict():

                    try:
                        csv_file = csv.reader(open('airport.csv', 'r'), delimiter=",")
                        for row in csv_file:
                            #Match code to equivalent row in CSV file and attain relevant data
                            if airport_code == str(row[4]):
                                cls.add_airport(country_currency, todays_currency_rates, airport_code,row[2],row[3],float(row[6]),float(row[7]))
         
                    except Exception as e:
                        print(Exception)
        
    
    
    
        #Distance Matrix
       
        #Check that each airport in list is in distance matrix dict and has a dict-matrix with each other airport in list
        for code in route_list:

            #If not in dict, create new nested dict to distance to each other airport
            if code not in cls.get_distance_matrix():
                sub_dist_dict = {}
                for code2 in route_list:
                    try:
                        #Use destination airport as key to inner nested dictionary in matrix
                        sub_dist_dict[code2] =  (cls.get_airport(code)).distance_between(cls.get_airport(code2))
                        
                    except:
                        print("Error: between:", code, code2)
                        #print(self.airport_dict[code].distance_between(self.airport_dict[code2]))
                        
                #Use departing airport as key to outer dictionary in matrix
                cls.add_distance(code, sub_dist_dict)

            #If airport is already a key in dictionary
            else:
                sub_dist_dict = {}
                for code2 in route_list:
                    #Add the other airports to the existing inner dict if they are not already part of it
                    if code2 not in cls.get_sub_distance_matrix(code):
                        try:
                            sub_dist_dict[code2] = (cls.get_airport(code)).distance_between(cls.get_airport(code2))
                        except:
                            print("Error: between:", code, code2)  
                            
                cls.append_sub_dict(code, sub_dist_dict)
        
        
        
        #Weighted Matrix
        
        #Create a dictionary for the weighted matrix based on todays currency rates
    
        #Same methodology from distance matrix above
        for code in route_list:
            if code not in cls.get_weight_matrix():
                cost_dict = {}
                for code2 in route_list:
                    try:
                        cost_dict[code2] = round((cls.get_distance_between(code, code2))*\
                                                 (cls.get_airport(code).get_currencyToday()),2)
                    except:
                        print("Error in weighted matrix between", code, code2)
                        
                #Add cost dictionary as the value to the airport_code as they key to the weighted_matrix       
                cls.add_weight(code, cost_dict)

            #If airport is already a key in dictionary
            else:
                cost_dict = {}
                for code2 in route_list:
                    if code2 not in cls.get_sub_weighted_matrix(code):
                        try:
                            cost_dict[code2] = round((cls.get_distance_between(code, code2))*\
                                                 (cls.get_airport(code).get_currencyToday()),2)
                        except:
                            print("Error in weighted matrix between", code, code2)
                            
                cls.append_sub_weight_dict(code, cost_dict)
     
    
        
    #Airport dict - Create a dictionary of Airport Codes : Actual Airport instances(objects) using Airport() Class
    @classmethod
    def add_airport(cls, country_currency, todays_currency_rates, airport_code, name, country, lat, long ):
        cls.__airport_dict[airport_code] = Airport(country_currency, todays_currency_rates, airport_code, name, country, lat, long)
    
    @classmethod
    def get_airport(cls, airport_code):
        return cls.__airport_dict[airport_code]
    
    @classmethod
    def get_airport_dict(cls):
        return cls.__airport_dict
    
    @classmethod
    def print_airport_dict(cls):
        for airport in cls.get_airport_dict():
            print(airport + " - " + str(cls.get_airport(airport)))
    
    
    
    #Dist Dict Methods
    @classmethod
    def add_distance(cls, airport_code, sub_dict):
        cls.__distance_matrix[airport_code] = sub_dict
        
    @classmethod
    def append_sub_dict(cls, airport_code, sub_dict):
        cls.__distance_matrix[airport_code].update(sub_dict)
        
    @classmethod
    def get_distance_matrix(cls):
        return cls.__distance_matrix
    
    @classmethod
    def get_sub_distance_matrix(cls, code):
        return cls.get_distance_matrix()[code]
    
    @classmethod
    def get_distance_between(cls, airport_code, to_airport):
        return cls.get_sub_distance_matrix(airport_code)[to_airport]
    
    @classmethod
    def print_distance_matrix(cls):
        print("DISTANCE MATRIX" + "\n************************************")
        for start_airport in cls.get_distance_matrix():
            print("DISTANCE FROM " + start_airport + " TO:")
            for dest_airport in cls.get_sub_distance_matrix(start_airport):
                print("\t"+dest_airport, str(cls.get_sub_distance_matrix(start_airport)[dest_airport]) + "km")
            print()
        print("END" + "\n************************************")
    
    
    
    #Weighted Dict Methods
    @classmethod
    def add_weight(cls, airport_code, sub_dict):
        cls.__weighted_matrix[airport_code] = sub_dict
    
    @classmethod
    def append_sub_weight_dict(cls, airport_code, sub_dict):
        cls.__weighted_matrix[airport_code].update(sub_dict)
        
    @classmethod
    def get_weight_matrix(cls):
        return cls.__weighted_matrix
    
    @classmethod
    def get_sub_weighted_matrix(cls, code):
        return cls.get_weight_matrix()[code]
    
    @classmethod
    def get_price_between(cls, airport_code, to_airport):
        return cls.get_sub_weighted_matrix(airport_code)[to_airport]
    
    @classmethod
    def print_weighed_matrix(cls):
        print("WEIGHTED MATRIX" + "\n************************************")
        for start_airport in cls.get_weight_matrix():
            print("PRICE FROM " + start_airport + " TO:")
            for dest_airport in cls.get_sub_weighted_matrix(start_airport):
                print("\t"+dest_airport, str(cls.get_sub_weighted_matrix(start_airport)[dest_airport]) + "e")
            print()
        print("END" + "\n************************************")

### Aircraft OBJECT

In [6]:
class aircraft():
    def __init__ (self, code, max_range):
        self.code = code
        self.max_range = max_range
    @property
    def code(self):
        return self.__code
    @code.setter
    def code(self, code):
        self.__code=code
    @property
    def max_range(self):
        return self.__max_range
    @max_range.setter
    def max_range(self, max_range):
        self.__max_range=max_range
    def display_aircraft(self):
        return('Code: '+self.code+". Max Range: "+str(self.max_range))

### Aircraft Roster OBJECT

In [7]:
class aircraft_roster():
    __aircraft_dict = {}
    try:
        with open('aircraft.csv') as __csvfile:
            __readCSV = csv.reader(__csvfile, delimiter=',')
            __csvfile.readline() #Skip first line
            for __row in __readCSV:
                #aircraft_dict[row[3]+" "+row[0]] = row[4]
                __aircraft_dict[__row[0]] = aircraft(__row[0], int(__row[4]))
        def __init__(self,name):
            self.name=name
    except FileNotFoundError as e:
        print("aircraft.csv file not found in current directory")

    @classmethod
    def __get_aircraftDict(cls):
        return cls.__aircraft_dict
    @classmethod
    def access_aircraft(cls, code):
        if code in cls.__aircraft_dict:
            return cls.__aircraft_dict[code]
        else:
            print('That aircraft is not currently part of the roster.')
    @classmethod        
    def add_aircraft(cls, code, max_range):
        if code not in cls.__aircraft_dict:
            cls.__aircraft_dict[code] = aircraft(code, int(max_range))
        else:
            print('That aircraft is already part of the roster.')
    @classmethod
    def remove(cls, code):
        if code in cls.__aircraft_dict:
            cls.__aircraft_dict.pop(code)
        else:
            print('That aircraft is not currently part of the roster.')
    @classmethod
    def display_roster(cls):
        __a=aircraft_roster.__get_aircraftDict()
        for plane in __a:
            print(__a[plane].display_aircraft())
    @classmethod
    def update_aircraft_max_range(cls, code, new_range):
        if code in cls.__aircraft_dict:
            aircraft_roster.access_aircraft(code).max_range=new_range
        else:
            print('That aircraft is not currently part of the roster.')     
    

<b>This class has a permutations method called recur. This is a brute force recursive method of finding all possible permutations given a list of airports. </b><br>    
This method is a recursive function which starts at the home airport(root) and finds all possible routes from this airport to all airports in a given list. As it creates the route list it also calculates the cost to travel from each airport to the next. The flow of the method can be visualised like a tree structure as shown below with the addition of root added to the end of each branch.

![title](tree.png)

This permutations function takes in a set of airports. When an element is removed from the set it is not replaced i.e a power set. <b>This means our function has O(n!).</b> This is a bad time complexity and as the input set grows the response time grows rapidly. After discussion, we decided that we could improve time complexity by caching subbranches using the root and next two nodes as a key (e.g. root-1-2). When this key recurs(e.g. root-1-2) or the reverse of this key occurs(e.g. root-2-1), we know that their subbranches will be the same.

### Best Route ALGORITHM

In [8]:
#Class that returns best route and price
class best_route():    
    
    def __init__(self, airportsToVisit, plane_code, aircraft_roster, atlas):
        try:
            #redirects printed error messages to devnull file rather than screen.
            atlas.update(airportsToVisit)

            #changes the plane_code to a string if it is entered as an int.
            if isinstance(plane_code, int):
                plane_code=str(plane_code)

            #checks to see if every airportcode given is valid.
            for code in airportsToVisit:
                atlas.get_airport(code)

            #checks to see if the plane code is part of the roster
            aircraft_roster.access_aircraft(plane_code).max_range

            #checks to ensure the home airport is also the final airport to be visited
            if airportsToVisit[0]!=airportsToVisit[-1]:
                airportsToVisit.append(airportsToVisit[0])
            test=airportsToVisit[1:-1]

            #checks for duplicate entries in the list. Removes them if there are.
            duplicates=[]
            for airport in test:
                count=0
                for airports in test:
                    if airport==airports:
                        count+=1
                if count>1:
                    if airport not in duplicates:
                        duplicates+=airport,
            if len(duplicates)>0:
                for airport in duplicates:
                    airportsToVisit.remove(airport)
                print("You have a duplicate airport(s) within your flight list. Removing the dulicates from the list." )
                print("New list of airports to calculate is "+str(airportsToVisit))

        except KeyError as e:
            print('Best Route cannot be instansiated because '+ str(e) +" is not in the atlas.")
        except AttributeError as e:
            print('That is not a valid plane code.')

        #if no errors are raised the new best_route object is made.
        else:
            self.__allRoutes=self.__recur(airportsToVisit,0, plane_code)
            self.__bestRoute=self.__cheapest_route(self.__allRoutes)

    #recurssive function that returns a list of the prices and routes of each possible combination   
    def __recur(self, myList, i, plane_code):
        #needed to calculate the number of routes in a branch below the current
        def fact(n, a): 
            if (n == 0):
                return a 
            return fact(n - 1, n * a)  
        #create a new list without the start airport for each trip
        total = 0
        working = myList[i]
        currentList = myList.copy()
        currentList.remove(working)

        #base case of recurssive function, e.g last trip of the journey, when only two possible airports remaining
        if len(myList) == 2:
            #holds the total price of trip calculated in the first list. Holds the airport codes it has travelled to in the second. 
            totals = [[],[]]
            totals[1].append(working+'-'+currentList[0])
            #if the plane cannot make the trip add False to the list instead of the price and airport code.
            if atlas.get_distance_between(working, currentList[0])>aircraft_roster.access_aircraft(plane_code).max_range:
                totals[0].append(False)
            else:
                price = atlas.get_price_between(working, currentList[0])
                totals[0].append(price)

        else:
            totals = [[],[]]
            #List of all other airports except departing airport and home airport
            for j in range(0,len(currentList)-1,+1):
                #Check to see if distance is within the plane range
                #if the plane cannot make the trip, there is no need to enter the recursive function to calculate
                #the routes below as we know the plane can't take that route.
                #By finding the factorial of the currentlist-2 you add the correct amount of False values 
                #to the list instead of the prices and airport codes.
                if atlas.get_distance_between(working, currentList[j])>aircraft_roster.access_aircraft(plane_code).max_range:
                    #print(working,"",currentList[j], "is impossible")
                    for k in range(0, fact(len(currentList)-2, 1)):
                        totals[0].append(False)
                        totals[1].append('False')
                else:
                    price = atlas.get_price_between(working, currentList[j])
                    #print(working,"",currentList[j], "is possible")
                    #x will hold the totals lists containing all possible routes and corresponding prices
                    #down the tree from this airport.
                    x=self.__recur(currentList, j, plane_code)
                    if x is not None:
                        for index in range(0, len(x[0])):
                            #k represents each price in the totals list from the trees below.
                            #if a branch below cannot be flown i.e price==False, the whole route must be false
                            if x[0][index]==False:
                                totals[0].append(False)
                                totals[1].append('False')
                            else:
                                #Adds the cost of the flights between the airports in the brances below 
                                #to the cost between the airports at this level
                                total = x[0][index] + price
                                #Adds the airport code of the airports flown in the brances below 
                                #to the current airport code
                                route = working +'-'+x[1][index]
                                #Add each possible combination to this airports list of possible combinations/tree
                                totals[0].append(total)
                                totals[1].append(route)
        #print(totals)
        return totals
    
    def display_allRoutes(self):
        for route in range(0, len(self.__allRoutes[0])):
            print(str(self.__allRoutes[1][route])+" : "+str(self.__allRoutes[0][route]))
            print()
    
    #Calcualtes the cheapest route using the mapped dictionary
    def __cheapest_route(self, mappedDict):  
        bestPrice = mappedDict[0][0]
        bestRoute = mappedDict[1][0]
        #Access list by index and compare price to min price and reassign accordingly

        for index in range(0, len(mappedDict[0])):
            #if bestPrice is false, replace with first non False price
            if bestPrice==False and mappedDict[0][index] != False:
                bestPrice = mappedDict[0][index]
                bestRoute = mappedDict[1][index]                
                                
            #if mappedDict[key][Price]not equal to false and bestPrice not equal to False, compare
            elif bestPrice!=False and mappedDict[0][index] != False:
                if mappedDict[0][index] < bestPrice:
                    bestPrice = mappedDict[0][index]
                    bestRoute = mappedDict[1][index]

        if bestPrice==False:
            return "There is no route that this plane can complete with it's fuel limitation"
        return str(bestRoute) + " is the cheapest route and costs " + str(bestPrice) + "."
    
    def get_cheapestRoute(self):
        return self.__bestRoute

## Instantiate all the objects once to represent one company using this software

In [10]:
cc = country_currency("16-05-19")
#country_currency.country_currency_code_dict

tc = todays_currency_rates("16-05-19")
#todays_currency_rates.currency_code_rates_dict

a = atlas(cc, tc, "condor_atlas")

ar =aircraft_roster("condor_roster")

# Tests all routes in the Test.csv

In [11]:
with open('test.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        test_list = []
        for row in readCSV:
            route_list = []
            for i in range(0, len(row)-1, +1):
                route_list += [row[i]]
                plane = str(row[len(row)-1])
            #print(route_list, plane)
            test_list += [route_list]
            test_list += [plane]
        #print(test_list)
        
        for i in range(0, len(test_list), +2):
            atlas.update(test_list[i])
            try:
                print(best_route(test_list[i], test_list[i+1], ar, a).get_cheapestRoute())
            except AttributeError as e:
                pass

There is no route that this plane can complete with it's fuel limitation
SNN-MAN-SIN-CDG-ORK-SNN is the cheapest route and costs 27290.3.
BOS-ATL-DFW-SFO-ORD-BOS is the cheapest route and costs 10634.58.
DUB-LHR-MOS-CPH-HEL-DUB is the cheapest route and costs 10791.0.


In [12]:
testAirports = ['AMS', 'CDG', 'DUB', 'LHR', 'JFK', 'AMS']
atlas.update(testAirports)
r1=best_route(testAirports, 'A320', ar, a)

In [13]:
r1.get_cheapestRoute()

'AMS-CDG-LHR-JFK-DUB-AMS is the cheapest route and costs 12067.01.'

In [14]:
atlas.print_distance_matrix()

DISTANCE MATRIX
************************************
DISTANCE FROM DUB TO:
	DUB 0.0km
	LHR 449.0km
	SYD 17215.0km
	JFK 5103.0km
	AAL 1097.0km
	CPH 1242.0km
	HEL 2023.0km
	MOS 1343.0km
	AMS 750.0km
	CDG 785.0km

DISTANCE FROM LHR TO:
	DUB 449.0km
	LHR 0.0km
	SYD 17020.0km
	JFK 5539.0km
	AAL 914.0km
	CPH 979.0km
	HEL 1847.0km
	MOS 897.0km
	AMS 370.0km
	CDG 348.0km

DISTANCE FROM SYD TO:
	DUB 17215.0km
	LHR 17020.0km
	SYD 0.0km
	JFK 16014.0km
	AAL 16140.0km

DISTANCE FROM JFK TO:
	DUB 5103.0km
	LHR 5539.0km
	SYD 16014.0km
	JFK 0.0km
	AAL 5967.0km
	AMS 5848.0km
	CDG 5834.0km

DISTANCE FROM AAL TO:
	DUB 1097.0km
	LHR 914.0km
	SYD 16140.0km
	JFK 5967.0km
	AAL 0.0km

DISTANCE FROM SNN TO:
	SNN 0.0km
	ORK 100.0km
	MAN 450.0km
	CDG 902.0km
	SIN 11402.0km

DISTANCE FROM ORK TO:
	SNN 100.0km
	ORK 0.0km
	MAN 452.0km
	CDG 842.0km
	SIN 11406.0km

DISTANCE FROM MAN TO:
	SNN 450.0km
	ORK 452.0km
	MAN 0.0km
	CDG 588.0km
	SIN 10956.0km

DISTANCE FROM CDG TO:
	SNN 902.0km
	ORK 842.0km
	MAN 588.0km
	CDG 0

In [15]:
with open('test.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        test_list = []
        for row in readCSV:
            route_list = []
            for i in range(0, len(row)-1, +1):
                route_list += [row[i]]
                plane = str(row[len(row)-1])
            #print(route_list, plane)
            test_list += [route_list]
            test_list += [plane]
        #print(test_list)
        
        for i in range(0, len(test_list), +2):
            atlas.update(test_list[i])
            try:
                print(best_route(test_list[i], test_list[i+1], ar, a).get_cheapestRoute())
            except AttributeError as e:
                pass

There is no route that this plane can complete with it's fuel limitation
SNN-MAN-SIN-CDG-ORK-SNN is the cheapest route and costs 27290.3.
BOS-ATL-DFW-SFO-ORD-BOS is the cheapest route and costs 10634.58.
DUB-LHR-MOS-CPH-HEL-DUB is the cheapest route and costs 10791.0.


In [16]:
atlas.update(["DUB", "WAS", "BRU"])

In [17]:
atlas.print_distance_matrix()

DISTANCE MATRIX
************************************
DISTANCE FROM DUB TO:
	DUB 0.0km
	LHR 449.0km
	SYD 17215.0km
	JFK 5103.0km
	AAL 1097.0km
	CPH 1242.0km
	HEL 2023.0km
	MOS 1343.0km
	AMS 750.0km
	CDG 785.0km
	WAS 5441.0km
	BRU 784.0km

DISTANCE FROM LHR TO:
	DUB 449.0km
	LHR 0.0km
	SYD 17020.0km
	JFK 5539.0km
	AAL 914.0km
	CPH 979.0km
	HEL 1847.0km
	MOS 897.0km
	AMS 370.0km
	CDG 348.0km

DISTANCE FROM SYD TO:
	DUB 17215.0km
	LHR 17020.0km
	SYD 0.0km
	JFK 16014.0km
	AAL 16140.0km

DISTANCE FROM JFK TO:
	DUB 5103.0km
	LHR 5539.0km
	SYD 16014.0km
	JFK 0.0km
	AAL 5967.0km
	AMS 5848.0km
	CDG 5834.0km

DISTANCE FROM AAL TO:
	DUB 1097.0km
	LHR 914.0km
	SYD 16140.0km
	JFK 5967.0km
	AAL 0.0km

DISTANCE FROM SNN TO:
	SNN 0.0km
	ORK 100.0km
	MAN 450.0km
	CDG 902.0km
	SIN 11402.0km

DISTANCE FROM ORK TO:
	SNN 100.0km
	ORK 0.0km
	MAN 452.0km
	CDG 842.0km
	SIN 11406.0km

DISTANCE FROM MAN TO:
	SNN 450.0km
	ORK 452.0km
	MAN 0.0km
	CDG 588.0km
	SIN 10956.0km

DISTANCE FROM CDG TO:
	SNN 902.0km
	ORK 